In [9]:
import numpy as np
import pandas as pd
import requests
import json
import pickle
import math

In [ ]:
configs = pickle.load(open('productionalized/config.pkl', 'rb'))
complete_view_ratio = configs['complete_view_ratio']
view_threshold = configs['view_threshold']
factor = configs['factor']
initial_bias = configs['initial_bias']

In [11]:
sql_query = "select studentid, count(*) from reels.pinot_view where watchDuration >= " + str(view_threshold) + " group by studentId having count(*)>=10 limit 10000000"
students = prod_query(sql_query)
students = students['studentid'].values
len(students)

191515

In [12]:
idcg = [1]
till_now = 0
for i in range(1, 1001):
    till_now += (1/math.log2(i+1))
    idcg.append(till_now)

In [13]:
def get_ndcg(ar):
    n = len(ar)
    dcg = 0
    for i in range(n):
        if ar[i]==True:
            dcg += (1/math.log2(i+2))
    return np.round(dcg/idcg[sum(ar)],4)

In [19]:
scores = []
for sid in students[0:2]:
    sql_query = "select created_at, reelid, watchduration/videoduration>=" + str(complete_view_ratio) + " as completed from reels.pinot_view where studentId='"+str(sid)+"' order by created_at limit 10000"
    df = query_results(sql_query)
    df = df.drop_duplicates(subset=['reelid']).reset_index(drop=True)
    time_list = list(df['created_at'])
    view_list = list(df['completed'])
    i, j, n = 0, 0, len(time_list)
    while i<n:
        while j<n and time_list[j]-time_list[i]<600000:
            j+=1
        scores.append([sid, time_list[i], j-i, sum(view_list[i:j]), get_ndcg(view_list[i:j])])
        i=j
df_scores = pd.DataFrame(scores, columns=['studentid', 'created_at', 'session_count', 'session_complete_count', 'ndcg'])
# df_scores.to_csv('reels_scores.csv', index=False)

TypeError: unsupported operand type(s) for -: 'str' and 'str'